In [1]:
from joblib import dump, load

# Import dependencies
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score
from joblib import dump, load

In [2]:
import sqlite3
import pandas as pd

# Create a connection to the database
conn = sqlite3.connect("fraud_detection.db")

# Read the data from the database
X_train = pd.read_sql_query("SELECT * FROM X_train", conn)
y_train = pd.read_sql_query("SELECT * FROM y_train", conn)["isFraud"]
X_test = pd.read_sql_query("SELECT * FROM X_test", conn)
y_test = pd.read_sql_query("SELECT * FROM y_test", conn)["isFraud"]

# Close the connection
conn.close()

In [3]:
# Train and evaluate the XGBoost models:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

# Train the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, random_state=42)
xgb_model.fit(X_train, y_train, eval_metric='logloss')



C:\Users\khans\anaconda3\envs\pythonData\lib\site-packages\xgboost\sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)

In [4]:
# Evaluate the models
y_pred_xgb = xgb_model.predict(X_test)

In [5]:
print("\nXGBoost:")
print(classification_report(y_test, y_pred_xgb))
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("ROC AUC:", roc_auc_score(y_test, y_pred_xgb))


XGBoost:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    113866
           1       0.92      0.48      0.63      4242

    accuracy                           0.98    118108
   macro avg       0.95      0.74      0.81    118108
weighted avg       0.98      0.98      0.98    118108

Accuracy: 0.9798574186337928
ROC AUC: 0.7394484317086844


In [6]:
# Save the trained model to a file
dump(xgb_model, 'xgb_model.joblib')

['xgb_model.joblib']